In [1]:
import spotipy.util as util
import spotipy
import pandas as pd
import numpy as np
import json
import time
import sys
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from collections import OrderedDict
from math import log
import calmap

In [2]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

#plotly setup
init_notebook_mode(connected=True)

pd.set_option('display.max_colwidth', None)

# Data Retrieval

In [3]:
import ast
from typing import List
from os import listdir
def get_streamings(path: str = 'Spotify Data') -> List[dict]:
    
    files = ['Spotify Data/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings
original = get_streamings()

In [4]:
username = '9cjfei8bx9vq78vel4odrjd3y'
client_id ='f2018fb393c2424dba2844c31e8e2c18'
client_secret = 'f4bb9567dafc4625877be647542fd919'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played user-top-read playlist-read-private user-library-read user-follow-read'

token = util.prompt_for_user_token(username=username, 
scope=scope, 
client_id=client_id,   
client_secret=client_secret,     
redirect_uri=redirect_uri)

In [5]:

import requests

# write the function to get track_id
def get_id(trackName: str,artistName:str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    trackArtist = trackName+ " " + artistName
    params = [
    ('q',track_artist ),#q is the search query parameter
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 10)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None   

In [6]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [34]:
spotify_data["trackId"] = spotify_data.apply(lambda x: get_id(x["trackName"],
                                                               x["artistName"],
                                                                 token),axis=1)

In [14]:
tracks = (spotify_data['trackName'])


features_tot = pd.DataFrame()
for track in tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        features_tot[track] = features

'''
for track in tracks:
    song = get_id(track,token)
    sp = spotipy.Spotify(auth=token)
    features = sp.audio_features(tracks = [song])[0]
    if features is not None:
        my_feature = my_feature.append({"song_id":song,
                                    "energy":features['energy'], 
                                    "liveness":features['liveness'],
                                    "tempo":features['tempo'],
                                    "speechiness":features['speechiness'],
                                    "acousticness":features['acousticness'],
                                    "instrumentalness":features['instrumentalness'],
                                    "danceability":features['danceability'],
                                    "duration_ms":features['duration_ms'],
                                    "loudness":features['loudness'],
                                    "valence":features['valence'],
                                    "mode":features['mode'],
                                    "key":features["key"],
                                 },ignore_index=True)
    else:
        pass
'''

'\nfor track in tracks:\n    song = get_id(track,token)\n    sp = spotipy.Spotify(auth=token)\n    features = sp.audio_features(tracks = [song])[0]\n    if features is not None:\n        my_feature = my_feature.append({"song_id":song,\n                                    "energy":features[\'energy\'], \n                                    "liveness":features[\'liveness\'],\n                                    "tempo":features[\'tempo\'],\n                                    "speechiness":features[\'speechiness\'],\n                                    "acousticness":features[\'acousticness\'],\n                                    "instrumentalness":features[\'instrumentalness\'],\n                                    "danceability":features[\'danceability\'],\n                                    "duration_ms":features[\'duration_ms\'],\n                                    "loudness":features[\'loudness\'],\n                                    "valence":features[\'valence\'],\n           

In [7]:
features_tot = pd.read_pickle(r'C:\Users\hp\Documents\ProjectOne\Created_features\top_features.pkl')

In [9]:
spotify_data = pd.DataFrame(original)
#spotify_data
spotify_data = spotify_data.drop(spotify_data[spotify_data["msPlayed"]<=120000].index)
spotify_data = spotify_data.reset_index(drop= True)
spotify_data = spotify_data.drop(['endTime'],axis=1)
spotify_data = (pd.DataFrame(spotify_data.groupby(['trackName','artistName']).sum())).reset_index()
spotify_data

,trackName,artistName,msPlayed
0,'Round Midnight,Thelonious Monk,141731
1,01: How to Achieve Your Data Viz Goals - Featured Data Visualization by Lisa Charlotte Rost,Data Viz Today,334038
2,21 Guns,Green Day,560897
3,A Letter Home,Berlioz,139500
4,A Simple Song,Chris Stapleton,216013
...,...,...,...
706,illusions,Idealism,131324
707,inherent,UKDD,152641
708,kate's not here,girl in red,177327
709,we'll stay inside when it rains,Rook1e,130585


In [10]:
streamings = pd.merge(features_tot,spotify_data,how="left",left_on='trackName',right_on='trackName')
streamings = streamings.dropna()
streamings

,trackName,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artistName,msPlayed
0,One More Night,0.716,0.821,5,-3.435,0,0.0314,0.0558,0,0.0844,...,92.997,audio_features,6cpk00i5TxCqSeqNi2HuIe,spotify:track:6cpk00i5TxCqSeqNi2HuIe,https://api.spotify.com/v1/tracks/6cpk00i5TxCqSeqNi2HuIe,Maroon 5,219547,4,Maroon 5,4817449
1,In The End,0.556,0.864,3,-5.87,0,0.0584,0.00958,0,0.209,...,105.143,audio_features,60a0Rd6pjrkxjPbaKzXjfq,spotify:track:60a0Rd6pjrkxjPbaKzXjfq,https://api.spotify.com/v1/tracks/60a0Rd6pjrkxjPbaKzXjfq,Black Veil Brides,216880,4,Black Veil Brides,4605837
2,Basket Case,0.442,0.943,3,-3.205,1,0.0602,0.00293,8.71e-06,0.091,...,85.064,audio_features,6L89mwZXSOwYl76YXfX13s,spotify:track:6L89mwZXSOwYl76YXfX13s,https://api.spotify.com/v1/tracks/6L89mwZXSOwYl76YXfX13s,Green Day,181533,4,Green Day,4352798
3,Give Me Novacaine / She's a Rebel,0.312,0.949,9,-3.533,1,0.0784,0.00185,1.78e-05,0.0569,...,154.795,audio_features,11I3KKHv7g2pXiuqRDabRA,spotify:track:11I3KKHv7g2pXiuqRDabRA,https://api.spotify.com/v1/tracks/11I3KKHv7g2pXiuqRDabRA,Green Day,326200,4,Green Day,10941190
4,I Knew You Were Trouble.,0.622,0.469,3,-6.798,0,0.0363,0.00454,2.25e-06,0.0335,...,77.019,audio_features,6FB3v4YcR57y4tXFcdxI1E,spotify:track:6FB3v4YcR57y4tXFcdxI1E,https://api.spotify.com/v1/tracks/6FB3v4YcR57y4tXFcdxI1E,Taylor Swift,219720,4,Taylor Swift,4237565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Free Bird,0.168,0.894,0,-4.267,1,0.0601,0.0015,0.00896,0.0642,...,151.855,audio_features,6CosyyANTKr1WY6aEeENSw,spotify:track:6CosyyANTKr1WY6aEeENSw,https://api.spotify.com/v1/tracks/6CosyyANTKr1WY6aEeENSw,Lynyrd Skynyrd,547600,4,Lynyrd Skynyrd,15581530
70,Ritual,0.647,0.726,3,-4.389,0,0.0552,0.157,0,0.0844,...,114.996,audio_features,3M9Apu4OZfylLTFKvgEtKa,spotify:track:3M9Apu4OZfylLTFKvgEtKa,https://api.spotify.com/v1/tracks/3M9Apu4OZfylLTFKvgEtKa,Black Veil Brides,198996,4,Black Veil Brides,4272626
71,Seeing Blind,0.716,0.555,9,-7.57,1,0.0368,0.0395,0,0.105,...,93.974,audio_features,65wnZsZLKJimfK1cTFOfa1,spotify:track:65wnZsZLKJimfK1cTFOfa1,https://api.spotify.com/v1/tracks/65wnZsZLKJimfK1cTFOfa1,Niall Horan,185281,4,Niall Horan,7983983
72,Story of My Life,0.6,0.663,3,-5.802,1,0.0477,0.225,0,0.119,...,121.07,audio_features,4nVBt6MZDDP6tRVdQTgxJg,spotify:track:4nVBt6MZDDP6tRVdQTgxJg,https://api.spotify.com/v1/tracks/4nVBt6MZDDP6tRVdQTgxJg,One Direction,245493,4,One Direction,9115959


In [13]:
means = pd.DataFrame(streamings.mean())
means = means.reset_index()

In [19]:
means = means.iloc[0:10,:]
means

,index,0
0,danceability,0.604649
1,energy,0.717486
2,key,5.081081
3,loudness,-5.725108
4,mode,0.554054
5,speechiness,0.068339
6,acousticness,0.143094
7,instrumentalness,0.012891
8,liveness,0.172351
9,valence,0.567164


In [20]:
fig = px.bar(means, x ='index', y = means[0])
fig.show()